# SOA - TP3 - Parte 3

Para este ejercicio se realizarán practicas con GPGPU. En esta parte, se planificarán los kernels del GPU, con hilos sobre $1$ multi-dimensión. El tema que se utilizará es la suma de dos vectores. El algoritmo está basado en la función axpy nivel 1[3], de la biblioteca BLAS[4] que resuelve la ecuación:

<center>$Y=\alpha X + Y$</center>

Su objetivo es enseñar a los alumnos como se utiliza Python [2] la plataforma Colab[1] y CUDA[5,6]. Mostrando el funcionamiento y granularidad (grilla, bloque, warps) de sobre una dimensión (x).

### 3.1.1. Preguntas del ejercicio
a) El programa del punto 3.1.2 se encuentra dividido en bloques, que se identifican con nombres de colores. El problema consiste en ordenar los bloques, ya que se encuentran desordenados. Para ello, haga foco en cada celda y utilice las flechas $\uparrow$ y $\downarrow$, para reorganizar los bloques según el orden correcto.

Nombre del bloque | Orden correcto
------------------|----------------
Verde             | 
Rosa              | 
Blanco            | 
Amarillo          | 
Gris              | 
Naranja           | 
Violeta           | 
Negro             | 
Rojo              | 
Azul              | 


*Tips:* Para ejecutar todas las celdas, ocúltelas con la flecha $\downarrow$ que está al lado del título "3.1.2 Ejecución del programa".

b) *Punto opcional:* Para verificar que el cálculo en GPU (junto con sus etapas) es correcto. Desarrolle la función axpy en forma secuencial (en lenguaje python) y compare los vectores resultados. 

---

**NOTA**: Si ya verificó que todo funciona bien continue:

c) Realice pruebas para diferentes valores de ```cantidad_N``` (500, 5.000, 50.000, 500.000 y 5.000.000). Compare los resultados con los obtenidos en el ejercicio 1.2.1 (OpenMP y secuencial).

  Tamaño | Tiempo secuencial | Tiempo openmp | Tiempo GPU
---------|-------------------|---------------|------------
     500 |                   |               |
   5.000 |                   |               |
  50.000 |                   |               |
 500.000 |                   |               |
5.000.000|                   |               |

d) A la comparación anterior, incluya como impactan el SpeedUP y Eficiencia al usar GPU.

e) A la comparación anterior, analice como fueron aumentando las dimensiones. ¿Se desperdiciaron muchos hilos? ¿Cómo se podría instanciar una cantidad de hilos exactamente igual al tamaño del vector?

f) ¿Cómo hace el programa, para almacenar el resultado en el vector ```r_cpu```, si este vector nunca se parametriza en el kernel?

g) Compare la función del kernel con la función ```axpy()``` del ejercicio 1.2.2 (OpenMP). ¿Qué cambios nota en el algoritmo? ¿El índice funciona igual? ¿Qué paso con el ciclo for en la función del kernel?


---
### 3.1.2 Ejecución del programa

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque VERDE

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, y_gpu )

# Fin bloque VERDE
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque ROSA

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# Fin bloque ROSA
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque BLANCO

# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = int( (cantidad_N+dim_hilo-1) / dim_hilo )

# Fin bloque BLANCO
# ------------------------------------------------------------------------------

Thread x:  256 , Bloque x: 1954


In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque AMARILLO

# #@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   500000#@param {type: "number"}
alfa =   1#@param {type: "number"}

# Fin bloque AMARILLO
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque GRIS

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )
y_gpu = cuda.mem_alloc( y_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )
cuda.memcpy_htod( y_gpu, y_cpu )

# Fin bloque GRIS
# ------------------------------------------------------------------------------

In [ ]:
#---------------------------
# Inicio bloque NARANJA
#
#  Armado del ambiente (Ojo)

!pip install pycuda

# Fin bloque NARANJA
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque VIOLETA

x_cpu = numpy.random.randn( cantidad_N )
x_cpu = x_cpu.astype( numpy.float32() )

y_cpu = numpy.random.randn( cantidad_N )
y_cpu = y_cpu.astype( numpy.float32() )

r_cpu = numpy.empty_like( x_cpu )

# Fin bloque VIOLETA
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque NEGRO

tiempo_total = datetime.now() - tiempo_total

print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print( "Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print( "Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

# Fin bloque NEGRO
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio bloque ROJO

tiempo_gpu = datetime.now()

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N),numpy.float32(alfa), x_gpu, y_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# Fin bloque ROJO
# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
# Inicio del bloque AZUL

module = SourceModule("""
__global__ void kernel_axpy( int n, float alfa, float *X, float *Y )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    Y[idx]  = alfa*X[idx] + Y[idx];
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_axpy")

# Fin del bloque AZUL
# ------------------------------------------------------------------------------

---
# Bibliografia

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[6] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)


